# Project Title: Fantasy Baseball 2017 Hitting Advanced Metrics Exploration
Project goal: Uncover undervalued players, make predictions for 2018 statistics

Advanced metrics were first collected by MLB in 2015 meaning I'll use 2015, 2016, 2017 data.

I initially selected a minimum of 50 ABs but seeing mostly pitchers under 75 ABs, I decided to make 75 ABs the minimum.

This notebook will be for cleaning and merging data from 2015, 2016, and 2017 from MLB Baseball Statcast and FanGraphs

# MLB Statcast Data Dictionary:

* pitches - ?
* player_id - unique player id, will be useful for year to year comparisons
* player_name - player's name
* total_pitches - number of pitches thrown
* pitch percent - ?
* ba - batting average
* iso - isolated power
* babip - batting average on balls in play
* slg - slugging percentage
* woba - weighted on-base average
* xwoba - expected weighted on-base average
* xba - expected batting average
* hits - total base hits
* abs - total at bats
* launch_speed - average launch speed off of bat
* launch_angle - average launch angle
* spin_rate - average spin rate 
* velocity - average velocity
* effective_speed - average effective velocity
* whiffs - how many swings and misses
* swings - number of swings
* takes - number of pitches taken
* eff_min_vel - difference between velocity and effective_speed
* release_extension - release extension
* posX_int_start_distance - starting distance from each position in the field


# FanGraphs Data Dictionary:

* Name - player name
* Team - team player plays for
* G - Games appeared in
* AB - At bats
* PA - Plate appearances
* HR - Home Runs
* R - Runs
* RBI - Runs batted in
* SB - Stolen bases
* BB% - Walks per plate appearance
* K% - Strikeouts per plate appearance
* ISO - Isolated slugging percentage
* BABIP - Batting average on balls in play
* OBP - On base percentage
* SLG - Slugging percentage
* wOBA - Weighted on base average
* wRC+ - Weighted runs created plus
* BsR - Baserunning
* Off - Offensive runs above average
* Def - Defensive runs aboe average
* WAR - Wins above replacement
* playerid - unique player id

In [147]:
# import packages
import pandas as pd
pd.options.display.max_columns = None

In [148]:
# read in 2015, 2016, and 2017 baseball statcast data
statcast_2015 = pd.read_csv("C:/Users/avitosky/Documents/Baseball Project/statcast_2015.csv")
statcast_2016 = pd.read_csv("C:/Users/avitosky/Documents/Baseball Project/statcast_2016.csv")
statcast_2017 = pd.read_csv("C:/Users/avitosky/Documents/Baseball Project/statcast_2017.csv")

In [149]:
# sort players by alphabetical order for indexing
statcast_2015 = statcast_2015.sort_values(by='player_name')
statcast_2016 = statcast_2016.sort_values(by='player_name')
statcast_2017 = statcast_2017.sort_values(by='player_name')

In [150]:
# reset index in savant data for merging later
statcast_2015 = statcast_2015.reset_index(drop=True)
statcast_2016 = statcast_2016.reset_index(drop=True)
statcast_2017 = statcast_2017.reset_index(drop=True)

In [151]:
# look at background info, mostly floats and ints with one object (player_name)
statcast_2015.info()
print("-------------------------------------")
statcast_2016.info()
print("-------------------------------------")
statcast_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 467 entries, 0 to 466
Data columns (total 31 columns):
pitches                    467 non-null int64
player_id                  467 non-null int64
player_name                467 non-null object
total_pitches              467 non-null int64
pitch_percent              467 non-null float64
ba                         467 non-null float64
iso                        467 non-null float64
babip                      467 non-null float64
slg                        467 non-null float64
woba                       467 non-null float64
xwoba                      467 non-null float64
xba                        467 non-null float64
hits                       467 non-null int64
abs                        467 non-null int64
launch_speed               467 non-null float64
launch_angle               467 non-null float64
spin_rate                  467 non-null int64
velocity                   467 non-null float64
effective_speed            467 non-null floa

In [152]:
# describe the 2015 data
statcast_2015.describe()

,pitches,player_id,total_pitches,pitch_percent,ba,iso,babip,slg,woba,xwoba,xba,hits,abs,launch_speed,launch_angle,spin_rate,velocity,effective_speed,whiffs,swings,takes,eff_min_vel,release_extension,pos3_int_start_distance,pos4_int_start_distance,pos5_int_start_distance,pos6_int_start_distance,pos7_int_start_distance,pos8_int_start_distance,pos9_int_start_distance
count,467.000000,467.000000,467.000000,467.0,467.000000,467.000000,467.000000,467.000000,467.000000,467.000000,467.000000,467.000000,467.000000,467.000000,467.000000,467.000000,467.000000,467.000000,467.000000,467.000000,467.000000,467.000000,467.000000,467.000000,467.000000,467.000000,467.000000,467.000000,467.000000,467.000000
mean,1422.134904,496932.550321,1422.134904,100.0,0.249657,0.148099,0.296062,0.397730,0.310516,0.304615,0.241713,86.770878,334.391863,86.553747,11.351392,2126.927195,88.722270,88.100128,150.158458,663.057816,743.873662,-0.621413,6.014518,109.160600,148.817987,109.513919,146.278373,291.475375,311.468951,291.321199
std,737.701078,81031.114490,737.701078,0.0,0.037963,0.059756,0.041732,0.080086,0.046327,0.044904,0.031366,50.148032,171.225984,2.722148,3.890019,30.290494,0.573724,0.617216,86.184828,342.287042,402.578256,0.210588,0.087200,3.311106,3.424705,8.100583,1.834231,7.941642,5.267990,6.671286
min,278.000000,116338.000000,278.000000,100.0,0.125000,0.000000,0.181000,0.152000,0.177000,0.177000,0.139000,10.000000,75.000000,78.000000,-2.000000,2012.000000,87.000000,86.250000,14.000000,115.000000,128.000000,-1.200000,5.780000,96.000000,139.000000,85.000000,139.000000,266.000000,295.000000,271.000000
25%,772.000000,452454.500000,772.000000,100.0,0.228000,0.104000,0.267000,0.344500,0.283500,0.277000,0.222000,44.000000,179.000000,84.800000,8.950000,2110.000000,88.400000,87.720000,78.500000,356.500000,391.000000,-0.800000,5.960000,107.000000,147.000000,103.500000,145.000000,287.000000,308.000000,287.000000
50%,1415.000000,500208.000000,1415.000000,100.0,0.254000,0.147000,0.299000,0.400000,0.313000,0.302000,0.244000,84.000000,333.000000,86.900000,11.400000,2130.000000,88.700000,88.100000,137.000000,655.000000,709.000000,-0.600000,6.010000,109.000000,149.000000,111.000000,146.000000,293.000000,312.000000,292.000000
75%,2049.000000,545839.500000,2049.000000,100.0,0.276000,0.188000,0.324000,0.449000,0.339000,0.333000,0.261000,130.500000,484.000000,88.500000,14.000000,2147.000000,89.100000,88.530000,208.500000,955.000000,1033.000000,-0.500000,6.080000,112.000000,150.000000,115.000000,147.500000,297.000000,315.000000,296.000000
max,3021.000000,656941.000000,3021.000000,100.0,0.338000,0.356000,0.412000,0.649000,0.467000,0.446000,0.333000,205.000000,638.000000,95.400000,24.600000,2213.000000,90.700000,90.130000,469.000000,1372.000000,1848.000000,0.000000,6.290000,117.000000,169.000000,141.000000,152.000000,310.000000,326.000000,309.000000


In [153]:
# describe the 2016 data
statcast_2016.describe()

,pitches,player_id,total_pitches,pitch_percent,ba,iso,babip,slg,woba,xwoba,xba,hits,abs,launch_speed,launch_angle,spin_rate,velocity,effective_speed,whiffs,swings,takes,eff_min_vel,release_extension,pos3_int_start_distance,pos4_int_start_distance,pos5_int_start_distance,pos6_int_start_distance,pos7_int_start_distance,pos8_int_start_distance,pos9_int_start_distance
count,459.000000,459.000000,459.000000,459.0,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.000000,459.00000,459.000000,459.000000,459.000000
mean,1472.324619,512616.934641,1472.324619,100.0,0.250930,0.156190,0.296978,0.407109,0.316893,0.311919,0.243619,88.446623,339.559913,83.332462,15.766231,2196.888889,88.734641,88.200109,158.503268,681.028322,776.413943,-0.537473,6.053420,108.657952,149.771242,110.655773,146.12854,294.265795,315.760349,292.583878
std,758.356172,78968.790969,758.356172,0.0,0.036263,0.058502,0.041438,0.076767,0.043414,0.040891,0.029207,51.316839,172.690368,2.248195,3.853504,25.585431,0.572399,0.621646,91.808453,351.398655,415.714310,0.212263,0.060611,3.513946,3.895182,8.323378,1.91262,7.326087,4.788915,6.207039
min,283.000000,120074.000000,283.000000,100.0,0.094000,0.008000,0.135000,0.153000,0.111000,0.185000,0.156000,8.000000,75.000000,76.400000,4.000000,2122.000000,86.900000,86.370000,17.000000,128.000000,111.000000,-1.300000,5.850000,98.000000,140.000000,88.000000,140.00000,269.000000,302.000000,276.000000
25%,811.000000,456273.000000,811.000000,100.0,0.228000,0.114500,0.271000,0.352000,0.286000,0.282000,0.224000,42.000000,185.000000,81.800000,13.100000,2180.000000,88.400000,87.800000,87.500000,375.000000,417.000000,-0.700000,6.010000,106.000000,148.000000,104.500000,145.00000,290.000000,313.000000,288.000000
50%,1372.000000,518692.000000,1372.000000,100.0,0.253000,0.152000,0.298000,0.411000,0.320000,0.312000,0.245000,81.000000,325.000000,83.600000,15.800000,2197.000000,88.800000,88.220000,146.000000,631.000000,715.000000,-0.500000,6.050000,108.000000,149.000000,112.000000,146.00000,296.000000,316.000000,293.000000
75%,2108.000000,572156.500000,2108.000000,100.0,0.275000,0.194000,0.322500,0.458500,0.346000,0.339000,0.264000,129.500000,506.000000,84.900000,18.300000,2214.000000,89.100000,88.645000,218.000000,989.000000,1092.500000,-0.400000,6.095000,112.000000,151.000000,115.000000,148.00000,299.500000,319.000000,297.000000
max,3014.000000,666560.000000,3014.000000,100.0,0.348000,0.358000,0.411000,0.657000,0.430000,0.459000,0.340000,216.000000,672.000000,88.600000,30.500000,2268.000000,90.700000,90.620000,462.000000,1426.000000,1813.000000,0.200000,6.230000,118.000000,171.000000,142.000000,151.00000,309.000000,329.000000,310.000000


In [154]:
# describe the 2017 data
statcast_2017.describe()

,pitches,player_id,total_pitches,pitch_percent,ba,iso,babip,slg,woba,xwoba,xba,hits,abs,launch_speed,launch_angle,spin_rate,velocity,effective_speed,whiffs,swings,takes,eff_min_vel,release_extension,pos3_int_start_distance,pos4_int_start_distance,pos5_int_start_distance,pos6_int_start_distance,pos7_int_start_distance,pos8_int_start_distance,pos9_int_start_distance
count,456.000000,456.000000,456.000000,456.0,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000
mean,1497.885965,533524.054825,1497.885965,100.0,0.252110,0.165803,0.298447,0.417908,0.322906,0.310399,0.239596,89.015351,341.486842,81.481140,15.921053,2217.614035,88.639254,88.148202,165.611842,691.890351,791.500000,-0.489474,6.023575,109.572368,150.484649,111.054825,147.125000,295.041667,317.682018,293.769737
std,735.232199,74635.920683,735.232199,0.0,0.036177,0.061619,0.042386,0.080516,0.044641,0.040224,0.028484,48.910703,166.171115,2.117513,4.018572,30.625568,0.509832,0.544963,91.322906,339.266463,405.669567,0.146080,0.069959,3.637295,3.787746,7.402647,2.064815,7.036515,4.969112,6.122346
min,277.000000,134181.000000,277.000000,100.0,0.134000,0.027000,0.145000,0.203000,0.181000,0.173000,0.156000,11.000000,75.000000,74.000000,-1.600000,2110.000000,86.800000,86.120000,6.000000,122.000000,115.000000,-1.000000,5.750000,99.000000,142.000000,90.000000,142.000000,271.000000,303.000000,274.000000
25%,839.000000,462042.000000,839.000000,100.0,0.231000,0.120750,0.274000,0.366750,0.295000,0.283000,0.220000,44.000000,185.750000,80.100000,13.075000,2196.000000,88.300000,87.770000,92.750000,388.250000,432.250000,-0.600000,5.980000,107.000000,148.000000,106.000000,146.000000,290.000000,314.000000,289.000000
50%,1465.000000,543388.500000,1465.000000,100.0,0.255000,0.162000,0.299000,0.417500,0.323000,0.311000,0.242000,88.000000,339.500000,81.700000,15.900000,2219.000000,88.600000,88.150000,147.500000,682.000000,766.500000,-0.500000,6.020000,108.000000,150.000000,112.000000,147.000000,296.000000,318.000000,294.000000
75%,2117.750000,594784.500000,2117.750000,100.0,0.276250,0.207250,0.328000,0.471000,0.350000,0.336000,0.259250,128.250000,487.000000,83.000000,18.500000,2241.250000,88.925000,88.490000,233.250000,976.250000,1114.750000,-0.400000,6.070000,113.000000,151.000000,115.000000,149.000000,300.000000,321.000000,298.000000
max,3028.000000,664056.000000,3028.000000,100.0,0.346000,0.392000,0.420000,0.690000,0.451000,0.446000,0.322000,213.000000,662.000000,87.100000,33.100000,2294.000000,90.200000,89.710000,458.000000,1464.000000,1832.000000,-0.100000,6.210000,119.000000,167.000000,134.000000,153.000000,311.000000,331.000000,309.000000


It looks like we have all of the same columns in the dataset so I'll explore all columns in 2017 dataset.

I know that pitches, player_id, ba, iso, babip, slg, woba, xwoba, xba, hits, abs, whiffs, swings, takes are all relevant 
columns but I'm not sure about the others so I will explore them now.

From the data above total_pitches is a mirror of pitches and pitch_percent only has values of 100 so I'll focus on the others

In [155]:
# launch speed may or may not be relevant, not a ton of variation in std
statcast_2017['launch_speed'].describe()

count    456.000000
mean      81.481140
std        2.117513
min       74.000000
25%       80.100000
50%       81.700000
75%       83.000000
max       87.100000
Name: launch_speed, dtype: float64

In [156]:
# simple correlation shows some correlation with xwoba (the most "catch-all stat in savant data) will keep the variable for now
statcast_2017['launch_speed'].corr(statcast_2017['xwoba'])

0.64513908275437226

In [157]:
# launch angle is a good descriptor of consistency for well struck balls so we'll leave it, higher launch angle is better
statcast_2017['launch_angle'].describe()

count    456.000000
mean      15.921053
std        4.018572
min       -1.600000
25%       13.075000
50%       15.900000
75%       18.500000
max       33.100000
Name: launch_angle, dtype: float64

In [158]:
# not much correlation but a good varaible for us to keep
statcast_2017['launch_angle'].corr(statcast_2017['xwoba'])

0.22459659563113282

In [159]:
# spin rate has a smaller std meaning that it is a relatively stable variable, more useful in pitching metrics than hitting
statcast_2017['spin_rate'].describe()

count     456.000000
mean     2217.614035
std        30.625568
min      2110.000000
25%      2196.000000
50%      2219.000000
75%      2241.250000
max      2294.000000
Name: spin_rate, dtype: float64

In [160]:
# with zero correlation to woba, we can get rid of it
statcast_2017['spin_rate'].corr(statcast_2017['xwoba'])

-0.014802210007920551

In [161]:
# same story with velocity as with spin_rate
statcast_2017['velocity'].describe()

count    456.000000
mean      88.639254
std        0.509832
min       86.800000
25%       88.300000
50%       88.600000
75%       88.925000
max       90.200000
Name: velocity, dtype: float64

In [162]:
# again zero correlation so we'll drop it
statcast_2017['velocity'].corr(statcast_2017['xwoba'])

0.014388267190704058

In [163]:
# again same story with effective_speed as with spin_rate and velocity
statcast_2017['effective_speed'].describe()

count    456.000000
mean      88.148202
std        0.544963
min       86.120000
25%       87.770000
50%       88.150000
75%       88.490000
max       89.710000
Name: effective_speed, dtype: float64

In [164]:
# zero correlation so we'll drop it
statcast_2017['effective_speed'].corr(statcast_2017['xwoba'])

0.012618671935544475

In [165]:
# and again same story with eff_min_vel as with spin_rate, velocity, and effective_speed
statcast_2017['eff_min_vel'].describe()

count    456.000000
mean      -0.489474
std        0.146080
min       -1.000000
25%       -0.600000
50%       -0.500000
75%       -0.400000
max       -0.100000
Name: eff_min_vel, dtype: float64

In [166]:
# zero correlation so we'll drop it
statcast_2017['eff_min_vel'].corr(statcast_2017['xwoba'])

-0.014219173418573736

In [167]:
# and one more time, same story with eff_min_vel as with spin_rate, velocity, effective_speed, and eff_min_vel
statcast_2017['release_extension'].describe()

count    456.000000
mean       6.023575
std        0.069959
min        5.750000
25%        5.980000
50%        6.020000
75%        6.070000
max        6.210000
Name: release_extension, dtype: float64

In [168]:
# zero correlation so we'll drop it
statcast_2017['release_extension'].corr(statcast_2017['xwoba'])

-0.025664659465450955

In [169]:
# the posX_int_distance variables describe the depth of the fielders positioning for defensive player so we'll drop those too
# kept - pitches, player_id, player_name, ba, iso, babip, slg, woba, xwoba, xba, hits, abs, launch_speed, launch_angle, whiffs, swings, taken
# deleted - total_pitches, pitch_percent, spin_rate, velocity, effective_speed, eff_min_vel, release_extension, posX_int_distance variables

In [170]:
# drop specified columns from data
statcast_2015.drop(['total_pitches', 'pitches', 'pitch_percent', 'abs', 'spin_rate', 'velocity',
                  'effective_speed', 'eff_min_vel', 'release_extension', 'pos3_int_start_distance',
                  'pos4_int_start_distance', 'pos5_int_start_distance', 'pos6_int_start_distance', 
                  'pos7_int_start_distance', 'pos8_int_start_distance', 'pos9_int_start_distance',
                  'ba', 'iso', 'babip', 'slg', 'woba'], axis=1, inplace=True)
statcast_2016.drop(['total_pitches', 'pitches', 'pitch_percent', 'abs', 'spin_rate', 'velocity',
                  'effective_speed', 'eff_min_vel', 'release_extension', 'pos3_int_start_distance',
                  'pos4_int_start_distance', 'pos5_int_start_distance', 'pos6_int_start_distance', 
                  'pos7_int_start_distance', 'pos8_int_start_distance', 'pos9_int_start_distance',
                  'ba', 'iso', 'babip', 'slg', 'woba'], axis=1, inplace=True)
statcast_2017.drop(['total_pitches', 'pitches', 'pitch_percent', 'abs', 'spin_rate', 'velocity',
                  'effective_speed', 'eff_min_vel', 'release_extension', 'pos3_int_start_distance',
                  'pos4_int_start_distance', 'pos5_int_start_distance', 'pos6_int_start_distance', 
                  'pos7_int_start_distance', 'pos8_int_start_distance', 'pos9_int_start_distance',
                  'ba', 'iso', 'babip', 'slg', 'woba'], axis=1, inplace=True)

In [171]:
# show updated dataset
statcast_2017.head()

,player_id,player_name,xwoba,xba,hits,launch_speed,launch_angle,whiffs,swings,takes
0,454560,A.J. Ellis,0.273,0.197,30,80.8,16.5,52,270,365
1,572041,A.J. Pollock,0.331,0.265,113,82.9,10.7,139,737,995
2,571437,Aaron Altherr,0.330,0.244,101,83.3,14.3,212,708,878
3,543305,Aaron Hicks,0.335,0.233,80,83.0,16.5,154,579,896
4,592450,Aaron Judge,0.446,0.278,154,85.1,17.5,429,1228,1756


To get a more holistic profile for each player, we'll need to import data from FanGraphs, drop what columns are redundant and then merge the two datasets together.

In [172]:
# read in 2015, 2016, and 2017 fangraphs data
fangraphs_2015 = pd.read_csv("C:/Users/avitosky/Documents/Baseball Project/fangraphs_2015.csv")
fangraphs_2016 = pd.read_csv("C:/Users/avitosky/Documents/Baseball Project/fangraphs_2016.csv")
fangraphs_2017 = pd.read_csv("C:/Users/avitosky/Documents/Baseball Project/fangraphs_2017.csv")

In [173]:
# statcast data has minimum 75 ABs, fangraphs can only specify minimum PAs so set to 70 but included ABs in data
# need to drop < 75 ABs from each fangraphs dataset
fangraphs_2015 = fangraphs_2015[fangraphs_2015["AB"] >= 75]
fangraphs_2016 = fangraphs_2016[fangraphs_2016["AB"] >= 75]
fangraphs_2017 = fangraphs_2017[fangraphs_2017["AB"] >= 75]

In [174]:
# change fangraphs 'Name' column to 'player_name' to merge with statcast data
fangraphs_2015.rename(columns={'Name':'player_name'}, inplace=True)
fangraphs_2016.rename(columns={'Name':'player_name'}, inplace=True)
fangraphs_2017.rename(columns={'Name':'player_name'}, inplace=True)

In [175]:
# merge fangraphs and statcast data to see where we have mismatched names and then print the differences
common_2015 = fangraphs_2015.merge(statcast_2015,on=['player_name','player_name'])
fangraphs_2015[(~fangraphs_2015.player_name.isin(common_2015.player_name))]

,player_name,Team,G,AB,PA,HR,R,RBI,SB,BB%,K%,ISO,BABIP,AVG,OBP,SLG,wOBA,wRC+,BsR,Off,Def,WAR,playerid
28,Gregory Bird,Yankees,46,157,178,11,26,31,0,10.7 %,29.8 %,0.268,0.319,0.261,0.343,0.529,0.372,137,-0.2,7.5,-4.5,0.9,14131
169,Nori Aoki,Giants,93,355,392,5,42,26,14,7.7 %,6.4 %,0.093,0.298,0.287,0.353,0.380,0.326,109,-3.6,0.6,-0.5,1.3,13075
237,Nick Castellanos,Tigers,154,549,595,15,42,73,0,6.6 %,25.5 %,0.164,0.322,0.255,0.303,0.419,0.311,94,-7.3,-11.7,-9.0,-0.1,11737
287,Ivan De Jesus,Reds,76,201,222,4,15,28,0,8.6 %,24.8 %,0.129,0.315,0.244,0.311,0.373,0.301,88,-0.6,-3.8,-1.8,0.1,9886
446,John Mayberry,Mets,59,110,119,3,8,9,1,7.6 %,27.7 %,0.155,0.203,0.164,0.227,0.318,0.240,51,0.3,-6.5,0.2,-0.3,3390
473,Eric Young,- - -,53,85,94,0,16,5,6,6.4 %,19.1 %,0.094,0.194,0.153,0.217,0.247,0.204,23,2.1,-6.4,-1.5,-0.5,7158


In [176]:
# what 2015 names need to be changed to
statcast_2015[(~statcast_2015.player_name.isin(common_2015.player_name))]

,player_id,player_name,xwoba,xba,hits,launch_speed,launch_angle,whiffs,swings,takes
175,458913,Eric Young Jr.,0.237,0.190,13,79.9,2.3,36,157,168
196,595885,Greg Bird,0.379,0.259,41,91.2,20.3,93,339,405
212,474443,Ivan De Jesus Jr.,0.302,0.244,49,84.8,6.3,97,410,548
256,460055,John Mayberry Jr.,0.266,0.216,18,85.5,10.1,64,221,250
365,592206,Nicholas Castellanos,0.325,0.259,140,86.2,17.5,342,1152,1163
373,493114,Norichika Aoki,0.284,0.243,102,84.5,3.5,61,655,791


In [177]:
# same thing for 2016 data
common_2016 = fangraphs_2016.merge(statcast_2016,on=['player_name','player_name'])
fangraphs_2016[(~fangraphs_2016.player_name.isin(common_2016.player_name))]

,player_name,Team,G,AB,PA,HR,R,RBI,SB,BB%,K%,ISO,BABIP,AVG,OBP,SLG,wOBA,wRC+,BsR,Off,Def,WAR,playerid
80,Nick Castellanos,Tigers,110,411,447,18,54,58,1,6.3 %,24.8 %,0.212,0.345,0.285,0.331,0.496,0.350,118,-1.6,8.0,-5.4,1.8,11737
191,Nori Aoki,Mariners,118,417,467,4,63,28,7,7.3 %,9.6 %,0.106,0.309,0.283,0.349,0.388,0.325,106,0.7,4.2,-8.8,1.2,13075
324,Byung-ho Park,Twins,62,215,244,12,28,24,1,8.6 %,32.8 %,0.219,0.230,0.191,0.275,0.409,0.294,79,1.5,-4.6,-3.3,0.0,18497
334,Yulieski Gurriel,Astros,36,130,137,3,13,15,1,3.6 %,8.8 %,0.123,0.267,0.262,0.292,0.385,0.292,83,-0.6,-3.3,-0.2,0.1,19198
375,Ivan De Jesus,Reds,104,221,243,1,21,20,3,7.0 %,21.0 %,0.059,0.324,0.253,0.311,0.312,0.277,67,-0.4,-10.1,3.7,0.1,9886


In [178]:
# what 2016 names need to be changed to
statcast_2016[(~statcast_2016.player_name.isin(common_2016.player_name))]

,player_id,player_name,xwoba,xba,hits,launch_speed,launch_angle,whiffs,swings,takes
74,666560,ByungHo Park,0.341,0.224,41,84.1,14.6,169,477,560
193,474443,Ivan De Jesus Jr.,0.294,0.258,56,79.8,11.3,94,445,537
341,592206,Nicholas Castellanos,0.379,0.283,117,83.4,17.7,272,932,792
350,493114,Norichika Aoki,0.279,0.237,118,83.2,8.3,101,798,870
456,493329,Yuli Gurriel,0.293,0.259,34,84.1,15.4,45,242,193


In [179]:
# and again for 2017 data
common_2017 = fangraphs_2017.merge(statcast_2017,on=['player_name','player_name'])
fangraphs_2017[(~fangraphs_2017.player_name.isin(common_2017.player_name))]

,player_name,Team,G,AB,PA,HR,R,RBI,SB,BB%,K%,ISO,BABIP,AVG,OBP,SLG,wOBA,wRC+,BsR,Off,Def,WAR,playerid
51,Nick Delmonico,White Sox,43,141,166,9,25,23,2,13.9 %,18.7 %,0.220,0.277,0.262,0.373,0.482,0.369,132,-0.5,6.1,-2.2,1.0,13157
109,Yulieski Gurriel,Astros,139,529,564,18,69,75,3,3.9 %,11.0 %,0.187,0.308,0.299,0.332,0.486,0.344,118,0.9,13.6,-14.7,1.8,19198
127,Nick Castellanos,Tigers,157,614,665,26,73,101,4,6.2 %,21.4 %,0.218,0.313,0.272,0.320,0.490,0.341,111,-2.1,6.7,-12.9,1.7,11737
172,Eric Young,Angels,47,110,125,4,24,16,12,4.0 %,24.8 %,0.155,0.333,0.264,0.336,0.418,0.329,108,2.3,3.5,0.4,0.8,7158
244,Nori Aoki,- - -,110,336,374,5,48,35,10,7.8 %,11.8 %,0.116,0.301,0.277,0.335,0.393,0.315,97,-1.2,-2.8,-5.5,0.4,13075
294,Gregory Bird,Yankees,48,147,170,9,20,28,0,11.2 %,24.7 %,0.231,0.194,0.190,0.288,0.422,0.303,86,-2.2,-5.1,-4.5,-0.4,14131
399,J.T. Riddle,Marlins,70,228,247,3,20,31,0,4.9 %,20.2 %,0.105,0.300,0.250,0.282,0.355,0.270,64,-1.2,-12.6,4.8,0.0,17642
456,Cam Perkins,Phillies,42,88,97,1,9,8,0,5.2 %,23.7 %,0.091,0.227,0.182,0.237,0.273,0.225,32,-0.5,-9.0,-1.3,-0.7,13444


In [180]:
# what 2017 names need to be changed to
statcast_2017[(~statcast_2017.player_name.isin(common_2017.player_name))]

,player_id,player_name,xwoba,xba,hits,launch_speed,launch_angle,whiffs,swings,takes
76,573088,Cameron Perkins,0.256,0.210,16,81.4,9.8,39,159,187
157,458913,Eric Young Jr.,0.251,0.198,29,78.4,7.2,66,220,204
176,595885,Greg Bird,0.324,0.206,28,84.6,24.6,95,311,412
196,595375,JT Riddle,0.268,0.226,57,80.8,11.4,106,464,426
342,592206,Nicholas Castellanos,0.366,0.283,167,83.0,16.4,358,1333,1266
348,547170,Nicky Delmonico,0.325,0.228,37,77.3,15.2,78,312,365
351,493114,Norichika Aoki,0.291,0.251,93,82.5,11.1,97,671,783
452,493329,Yuli Gurriel,0.327,0.283,158,85.1,13.8,164,932,991


In [181]:
fangraphs_2015['player_name'] = fangraphs_2015['player_name'].replace({'Eric Young' : 'Eric Young Jr.', 
                                        'Gregory Bird': 'Greg Bird', 'Ivan De Jesus': 'Ivan De Jesus Jr.', 
                                        'John Mayberry': 'John Mayberry Jr.', 'Nick Castellanos': 'Nicholas Castellanos', 
                                        'Nori Aoki': 'Norichika Aoki'})
fangraphs_2016['player_name'] = fangraphs_2016['player_name'].replace({'Byung-ho Park': 'ByungHo Park',
                                        ' Ivan De Jesus': 'Ivan De Jesus Jr.', 'Nick Castellanos': 'Nicholas Castellanos', 
                                        'Nori Aoki': 'Norichika Aoki','Yulieski Gurriel': 'Yuli Gurriel'})
fangraphs_2017['player_name'] = fangraphs_2017['player_name'].replace({'Cam Perkins': 'Cameron Perkins',
                                        'Eric Young': 'Eric Young Jr.', 'Gregory Bird' : 'Greg Bird', 
                                        'J.T. Riddle': 'JT Riddle', 'Nick Castellanos': 'Nicholas Castellanos', 
                                        'Nick Delmonico': 'Nicky Delmonico', 'Nori Aoki': 'Norichika Aoki', 
                                        'Yulieski Gurriel': 'Yuli Gurriel'})

In [182]:
# sort fangraphs data by name
fangraphs_2015 = fangraphs_2015.sort_values(by='player_name')
fangraphs_2016 = fangraphs_2016.sort_values(by='player_name')
fangraphs_2017 = fangraphs_2017.sort_values(by='player_name')

In [183]:
# reset fangraphs index
fangraphs_2015 = fangraphs_2015.reset_index(drop=True)
fangraphs_2016 = fangraphs_2016.reset_index(drop=True)
fangraphs_2017 = fangraphs_2017.reset_index(drop=True)

In [184]:
# look at background info, mostly floats and ints with one object (player_name)
fangraphs_2015.info()
print("-------------------------------------")
fangraphs_2016.info()
print("-------------------------------------")
fangraphs_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 467 entries, 0 to 466
Data columns (total 23 columns):
player_name    467 non-null object
Team           467 non-null object
G              467 non-null int64
AB             467 non-null int64
PA             467 non-null int64
HR             467 non-null int64
R              467 non-null int64
RBI            467 non-null int64
SB             467 non-null int64
BB%            467 non-null object
K%             467 non-null object
ISO            467 non-null float64
BABIP          467 non-null float64
AVG            467 non-null float64
OBP            467 non-null float64
SLG            467 non-null float64
wOBA           467 non-null float64
wRC+           467 non-null int64
BsR            467 non-null float64
Off            467 non-null float64
Def            467 non-null float64
WAR            467 non-null float64
playerid       467 non-null int64
dtypes: float64(10), int64(9), object(4)
memory usage: 84.0+ KB
-------------------------

In [185]:
fangraphs_2017.head()

,player_name,Team,G,AB,PA,HR,R,RBI,SB,BB%,K%,ISO,BABIP,AVG,OBP,SLG,wOBA,wRC+,BsR,Off,Def,WAR,playerid
0,A.J. Ellis,Marlins,51,143,163,6,17,14,0,7.4 %,17.8 %,0.161,0.222,0.210,0.298,0.371,0.294,80,-1.8,-6.0,2.8,0.2,5677
1,A.J. Pollock,Diamondbacks,112,425,466,14,73,49,20,7.5 %,15.2 %,0.205,0.291,0.266,0.330,0.471,0.340,103,2.6,4.2,1.8,2.1,9256
2,Aaron Altherr,Phillies,107,372,412,19,58,65,5,7.8 %,25.2 %,0.245,0.328,0.272,0.340,0.516,0.359,120,-2.0,8.7,-8.6,1.3,11270
3,Aaron Hicks,Yankees,88,301,361,15,54,52,10,14.1 %,18.6 %,0.209,0.290,0.266,0.372,0.475,0.363,127,2.5,14.4,6.4,3.3,5297
4,Aaron Judge,Yankees,155,542,678,52,128,114,9,18.7 %,30.7 %,0.343,0.357,0.284,0.422,0.627,0.430,173,0.0,60.8,-1.3,8.2,15640


In [186]:
# need to convert BB% and K% objects to floats
fangraphs_2015 ['BB%'] = fangraphs_2015['BB%'].replace('%', ' ', regex=True).astype('float')/100
fangraphs_2016 ['BB%'] = fangraphs_2016['BB%'].replace('%', ' ', regex=True).astype('float')/100
fangraphs_2017 ['BB%'] = fangraphs_2017['BB%'].replace('%', ' ', regex=True).astype('float')/100
fangraphs_2015 ['K%'] = fangraphs_2015['K%'].replace('%', ' ', regex=True).astype('float')/100
fangraphs_2016 ['K%'] = fangraphs_2016['K%'].replace('%', ' ', regex=True).astype('float')/100
fangraphs_2017 ['K%'] = fangraphs_2017['K%'].replace('%', ' ', regex=True).astype('float')/100

In [187]:
# make sure data looks good after converting to floats
fangraphs_2017.head()

,player_name,Team,G,AB,PA,HR,R,RBI,SB,BB%,K%,ISO,BABIP,AVG,OBP,SLG,wOBA,wRC+,BsR,Off,Def,WAR,playerid
0,A.J. Ellis,Marlins,51,143,163,6,17,14,0,0.074,0.178,0.161,0.222,0.210,0.298,0.371,0.294,80,-1.8,-6.0,2.8,0.2,5677
1,A.J. Pollock,Diamondbacks,112,425,466,14,73,49,20,0.075,0.152,0.205,0.291,0.266,0.330,0.471,0.340,103,2.6,4.2,1.8,2.1,9256
2,Aaron Altherr,Phillies,107,372,412,19,58,65,5,0.078,0.252,0.245,0.328,0.272,0.340,0.516,0.359,120,-2.0,8.7,-8.6,1.3,11270
3,Aaron Hicks,Yankees,88,301,361,15,54,52,10,0.141,0.186,0.209,0.290,0.266,0.372,0.475,0.363,127,2.5,14.4,6.4,3.3,5297
4,Aaron Judge,Yankees,155,542,678,52,128,114,9,0.187,0.307,0.343,0.357,0.284,0.422,0.627,0.430,173,0.0,60.8,-1.3,8.2,15640


In [188]:
# from fangraphs will drop variables Team, G, BsR Off, Def (both are cumulative stats, not rate stats)
fangraphs_2015.drop(['Team', 'G', 'Off', 'Def', 'BsR'], axis=1, inplace=True)
fangraphs_2016.drop(['Team', 'G', 'Off', 'Def', 'BsR'], axis=1, inplace=True)
fangraphs_2017.drop(['Team', 'G', 'Off', 'Def', 'BsR'], axis=1, inplace=True)

In [189]:
# check to see what end of dataset looks like, hopefully we have same names and row count in fangraphs data!
statcast_2017.tail()

,player_id,player_name,xwoba,xba,hits,launch_speed,launch_angle,whiffs,swings,takes
451,475174,Yonder Alonso,0.368,0.262,120,83.0,20.4,255,976,1096
452,493329,Yuli Gurriel,0.327,0.283,158,85.1,13.8,164,932,991
453,488862,Yunel Escobar,0.330,0.272,96,84.3,7.1,126,637,718
454,446359,Zack Cozart,0.332,0.255,130,81.7,16.3,139,868,1249
455,643335,Zack Granite,0.291,0.250,22,79.9,15.4,10,160,265


In [190]:
# and we do! now time to merge
fangraphs_2017.tail()

,player_name,AB,PA,HR,R,RBI,SB,BB%,K%,ISO,BABIP,AVG,OBP,SLG,wOBA,wRC+,WAR,playerid
451,Yonder Alonso,451,521,28,72,67,2,0.131,0.226,0.235,0.302,0.266,0.365,0.501,0.366,132,2.4,2530
452,Yuli Gurriel,529,564,18,69,75,3,0.039,0.110,0.187,0.308,0.299,0.332,0.486,0.344,118,1.8,19198
453,Yunel Escobar,350,381,7,43,31,1,0.076,0.134,0.123,0.305,0.274,0.333,0.397,0.318,100,0.8,4191
454,Zack Cozart,438,507,24,80,63,3,0.122,0.154,0.251,0.312,0.297,0.385,0.548,0.392,141,5.0,2616
455,Zack Granite,93,107,1,14,13,2,0.112,0.084,0.054,0.250,0.237,0.321,0.290,0.278,67,-0.2,15343


In [191]:
# merge 2015 fangraphs and savant data on index
merged_2015 = pd.merge(fangraphs_2015, statcast_2015, right_index=True, left_index=True)
merged_2016 = pd.merge(fangraphs_2016, statcast_2016, right_index=True, left_index=True)
merged_2017 = pd.merge(fangraphs_2017, statcast_2017, right_index=True, left_index=True)

In [192]:
# let's look at the merged tail and make sure it merged correctly
merged_2017.tail()

,player_name_x,AB,PA,HR,R,RBI,SB,BB%,K%,ISO,BABIP,AVG,OBP,SLG,wOBA,wRC+,WAR,playerid,player_id,player_name_y,xwoba,xba,hits,launch_speed,launch_angle,whiffs,swings,takes
451,Yonder Alonso,451,521,28,72,67,2,0.131,0.226,0.235,0.302,0.266,0.365,0.501,0.366,132,2.4,2530,475174,Yonder Alonso,0.368,0.262,120,83.0,20.4,255,976,1096
452,Yuli Gurriel,529,564,18,69,75,3,0.039,0.110,0.187,0.308,0.299,0.332,0.486,0.344,118,1.8,19198,493329,Yuli Gurriel,0.327,0.283,158,85.1,13.8,164,932,991
453,Yunel Escobar,350,381,7,43,31,1,0.076,0.134,0.123,0.305,0.274,0.333,0.397,0.318,100,0.8,4191,488862,Yunel Escobar,0.330,0.272,96,84.3,7.1,126,637,718
454,Zack Cozart,438,507,24,80,63,3,0.122,0.154,0.251,0.312,0.297,0.385,0.548,0.392,141,5.0,2616,446359,Zack Cozart,0.332,0.255,130,81.7,16.3,139,868,1249
455,Zack Granite,93,107,1,14,13,2,0.112,0.084,0.054,0.250,0.237,0.321,0.290,0.278,67,-0.2,15343,643335,Zack Granite,0.291,0.250,22,79.9,15.4,10,160,265


In [193]:
# check in to see what our merged data looks like
merged_2015.info()
print("-------------------------------------")
merged_2016.info()
print("-------------------------------------")
merged_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 467 entries, 0 to 466
Data columns (total 28 columns):
player_name_x    467 non-null object
AB               467 non-null int64
PA               467 non-null int64
HR               467 non-null int64
R                467 non-null int64
RBI              467 non-null int64
SB               467 non-null int64
BB%              467 non-null float64
K%               467 non-null float64
ISO              467 non-null float64
BABIP            467 non-null float64
AVG              467 non-null float64
OBP              467 non-null float64
SLG              467 non-null float64
wOBA             467 non-null float64
wRC+             467 non-null int64
WAR              467 non-null float64
playerid         467 non-null int64
player_id        467 non-null int64
player_name_y    467 non-null object
xwoba            467 non-null float64
xba              467 non-null float64
hits             467 non-null int64
launch_speed     467 non-null float64
launc

In [194]:
# two player names columns after merging (short/full name differences) and player_id is now our index so we'll drop that too
merged_2015.drop(['player_name_y', 'player_id'], axis=1, inplace=True)
merged_2016.drop(['player_name_y', 'player_id'], axis=1, inplace=True)
merged_2017.drop(['player_name_y', 'player_id'], axis=1, inplace=True)

In [195]:
# making all column names consistent for readability moving forward
merged_2015.rename(columns={'player_name_x':'Player_Name', 'AVG':'BA', 'pitches':'Pitches', 'BB%':'BB/PA', 'K%':'K/PA',
                            'iso':'ISO', 'babip':'BABIP', 'slg':'SLG', 'woba':'wOBA', 'xwoba':'xwOBA', 'xba':'xBA', 
                            'hits':'Hits', 'launch_speed':'Launch_Speed', 'launch_angle':'Launch_Angle', 'whiffs':'Whiffs', 
                            'swings':'Swings', 'takes':'Takes'}, inplace=True)
merged_2016.rename(columns={'player_name_x':'Player_Name', 'AVG':'BA', 'pitches':'Pitches', 'BB%':'BB/PA', 'K%':'K/PA',
                            'iso':'ISO', 'babip':'BABIP', 'slg':'SLG', 'woba':'wOBA', 'xwoba':'xwOBA', 'xba':'xBA', 
                            'hits':'Hits', 'launch_speed':'Launch_Speed', 'launch_angle':'Launch_Angle', 'whiffs':'Whiffs', 
                            'swings':'Swings', 'takes':'Takes'}, inplace=True)
merged_2017.rename(columns={'player_name_x':'Player_Name', 'AVG':'BA', 'pitches':'Pitches', 'BB%':'BB/PA', 'K%':'K/PA',
                            'iso':'ISO', 'babip':'BABIP', 'slg':'SLG', 'woba':'wOBA', 'xwoba':'xwOBA', 'xba':'xBA', 
                            'hits':'Hits', 'launch_speed':'Launch_Speed', 'launch_angle':'Launch_Angle', 'whiffs':'Whiffs', 
                            'swings':'Swings', 'takes':'Takes'}, inplace=True)

In [196]:
# reorder columns for more logical ordering
merged_2015 = merged_2015[['Player_Name', 'PA', 'AB', 'Hits', 'R', 'HR', 'RBI', 
                           'SB', 'BA', 'xBA', 'OBP', 'BABIP', 'ISO', 'SLG', 'wOBA', 'xwOBA', 
                           'BB/PA', 'K/PA', 'Launch_Speed', 'Launch_Angle', 'Whiffs', 
                           'Swings', 'Takes', 'wRC+', 'WAR', 'playerid']]
merged_2016 = merged_2016[['Player_Name', 'PA', 'AB', 'Hits', 'R', 'HR', 'RBI', 
                           'SB', 'BA', 'xBA', 'OBP', 'BABIP', 'ISO', 'SLG', 'wOBA', 'xwOBA', 
                           'BB/PA', 'K/PA', 'Launch_Speed', 'Launch_Angle', 'Whiffs', 
                           'Swings', 'Takes', 'wRC+', 'WAR', 'playerid']]
merged_2017 = merged_2017[['Player_Name', 'PA', 'AB', 'Hits', 'R', 'HR', 'RBI', 
                           'SB', 'BA', 'xBA', 'OBP', 'BABIP', 'ISO', 'SLG', 'wOBA', 'xwOBA', 
                           'BB/PA', 'K/PA', 'Launch_Speed', 'Launch_Angle', 'Whiffs', 
                           'Swings', 'Takes', 'wRC+', 'WAR', 'playerid']]

In [197]:
# save merged files as csv for further analysis
merged_2015.to_csv("C:/Users/avitosky/Documents/Baseball Project/merged_2015.csv")
merged_2016.to_csv("C:/Users/avitosky/Documents/Baseball Project/merged_2016.csv")
merged_2017.to_csv("C:/Users/avitosky/Documents/Baseball Project/merged_2017.csv")